In [24]:
# 설치
!pip install -U langchain langchain-openai langchain-community langchain-text-splitters faiss-cpu PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.3 MB/s eta 0:00:00


In [25]:
# 1. OpenAI 키 설정
import os
os.environ["OPENAI_API_KEY"] = "..."


In [26]:
# 2. PDF 업로드
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]


Saving 2005.11401v4.pdf to 2005.11401v4 (3).pdf


In [27]:
# 3. 문서 로딩 (PyMuPDF)
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(pdf_path)
docs = loader.load()


In [28]:
# 4. 텍스트 분할
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
split_docs = splitter.split_documents(docs)


In [45]:
# 5. OpenAI Embedding + FAISS 저장
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embedding_model = OpenAIEmbeddings()
db = FAISS.from_documents(split_docs, embedding_model)

retriever = db.as_retriever(search_kwargs={"k": 15})


In [46]:
# 6. 요약 쿼리 → 핵심 청크 가져오기
query = "논문의 배경, 제안한 모델 구조, 실험 결과, 기여점에 대해 풍부하게 설명된 문단을 가져와줘"
relevant_docs = retriever.get_relevant_documents(query)
retrieved_context = "\n\n".join([doc.page_content for doc in relevant_docs])

retrieved_context[:1000]


'https://www.aclweb.org/anthology/P17-1171.\n[6] Eunsol Choi, Daniel Hewlett, Jakob Uszkoreit, Illia Polosukhin, Alexandre Lacoste, and\nJonathan Berant. Coarse-to-ﬁne question answering for long documents. In Proceedings of the\n55th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers),\npages 209–220, Vancouver, Canada, July 2017. Association for Computational Linguistics. doi:\n10.18653/v1/P17-1020. URL https://www.aclweb.org/anthology/P17-1020.\n10\n\nfor Computational Linguistics. doi: 10.18653/v1/N19-4009. URL https://www.aclweb.\norg/anthology/N19-4009.\n[46] Ethan Perez, Siddharth Karamcheti, Rob Fergus, Jason Weston, Douwe Kiela, and Kyunghyun\nCho. Finding generalizable evidence by learning to convince q&a models. In Proceedings\nof the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th\nInternational Joint Conference on Natural Language Processing (EMNLP-IJCNLP), pages\n2402–2411, Hong Kong, China, November 2019

In [47]:
# 7. 프롬프트 구성
prompt = f"""
다음은 논문에서 검색된 핵심 내용입니다.

이 논문을 항목별로 깊이 있고 중복 없이 다음과 같은 형식으로 **bullet point로** 요약해 주세요.
기술적 세부사항은 전부 포함되어야 하며, 실험 내용과 결과가 풍부해야 됩니다.
결론과 향후 연구 방향도 포함해 주세요.

- 📌 논문 제목:
  - ...
  - ...

- 🎯 문제 정의:
  - ...
  - ...

- 🧠 제안하는 모델 및 구조:
  - ...
  - ...

- 🔬 실험 및 결과:
  - ...
  - ...

- 📎 주요 기여점:
  - ...
  - ...

내용:
{retrieved_context}
"""


In [48]:
# 8. SystemMessage 추가 및 gpt-4o-mini 질의
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage,HumanMessage
messages = [
    SystemMessage(content="당신은 논문을 읽고 항목별로 풍부하고 구체적인 요약을 작성하는 AI입니다."),
    HumanMessage(content=prompt),
]
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

response = llm.invoke([HumanMessage(content=prompt)])
print("요약 결과:\n", response.content)


📄 요약 결과:
 - 📌 논문 제목:
  - "Coarse-to-fine question answering for long documents"
  - Eunsol Choi et al., 2017

- 🎯 문제 정의:
  - 긴 문서에서 질문에 대한 답변을 찾는 것이 어려움.
  - 기존의 QA 시스템은 긴 문서에서 정보 검색의 효율성이 떨어짐.
  - 문서의 모든 내용을 고려하지 않고 특정 부분만을 참조하여 답변을 생성하는 문제.

- 🧠 제안하는 모델 및 구조:
  - Coarse-to-fine 접근 방식을 사용하여 질문에 대한 답변을 단계적으로 생성.
  - 초기 단계에서 문서의 전체 내용을 고려하여 관련 문서를 검색하고, 이후 세부적인 정보를 추출.
  - RAG( Retrieval-Augmented Generation) 모델을 제안, 이는 사전 훈련된 언어 모델과 검색 모듈을 결합하여 정보 검색 및 생성 능력을 향상.

- 🔬 실험 및 결과:
  - 다양한 QA 작업에서 RAG 모델을 평가하여 기존의 seq2seq 모델 및 특정 작업에 최적화된 아키텍처보다 우수한 성능을 보임.
  - Jeopardy 질문 생성 실험에서 RAG 모델이 더 구체적이고 다양한 언어를 생성함.
  - FEVER 데이터셋에서 RAG 모델이 3-way 분류에서 최첨단 모델과 유사한 성능을 보임.
  - RAG의 검색 메커니즘이 모든 작업에서 성능 개선에 기여함을 입증.

- 📎 주요 기여점:
  - 긴 문서에서의 질문 응답을 위한 새로운 접근 방식 제안.
  - 검색 모듈과 생성 모델의 결합을 통해 정보 검색의 효율성을 높임.
  - 모델의 메모리가 인간이 읽을 수 있는 형태로 구성되어 해석 가능성과 동적 업데이트 가능성을 제공.

- 🔮 결론 및 향후 연구 방향:
  - RAG 모델이 다양한 QA 작업에서 효과적임을 입증하며, 향후 연구는 더 많은 NLP 작업에 대한 일반화 가능성을 탐구할 예정.
  - 검색 메커니즘의 최적화 및 다양한 문서 형식에 대한 적응력을 향상시키는 방

GPT 4o 완성도 점수

평가 항목 | 점수 (/10)
정보 정확도 | 10
기술적 깊이 | 9.5
논리 흐름 | 10
형식 가독성 | 10
실무 활용성 (슬라이드, 보고서 등) | 10